# Semantic similarity


In [10]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors

# Load GloVe vectors using gensim
# Assuming you have the GloVe vectors in the "glove.6B.100d.txt" format
glove_model = KeyedVectors.load_word2vec_format(
    "../data/external/glove.6B.100d.txt", binary=False, no_header=True
)


def get_sentence_vector(sentence, model):
    words = sentence.split()
    # Filter out words that are not in the model's key_to_index
    words = [word for word in words if word in model.key_to_index]
    # If no words of the sentence are in the model's key_to_index, return a zero vector
    if len(words) == 0:
        return np.zeros(model.vector_size)
    # Get vectors for each word and average them
    vectors = [model[word] for word in words]
    sentence_vector = np.mean(vectors, axis=0)
    return sentence_vector


def compute_semantic_similarity(sent1, sent2, model):
    vec1 = get_sentence_vector(sent1, model)
    vec2 = get_sentence_vector(sent2, model)
    return cosine_similarity([vec1], [vec2])[0][0]


# Test
original_sentence = "This is an example sentence."
paraphrased_sentence = "This sentence is an example."

similarity = compute_semantic_similarity(
    original_sentence, paraphrased_sentence, glove_model
)
print("Semantic Similarity:", similarity)

Semantic Similarity: 0.9176121


# Discriminator network (deprecated, not used)


In [ ]:
from transformers import (
    BertTokenizer,
    BertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
import torch

# 1. Load the dataset
# Assuming you have a dataset in the following format:
# original_text, paraphrased_text, label (1 for human, 0 for machine)
data = pd.read_csv("labeled_dataset.csv")

# Tokenize the data
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


def encode_sentences(original, paraphrase):
    return tokenizer(
        original,
        paraphrase,
        truncation=True,
        padding="max_length",
        max_length=128,
        return_tensors="pt",
    )


dataset = data.apply(
    lambda x: encode_sentences(x["original_text"], x["paraphrased_text"]), axis=1
)

# Convert labels to tensors
labels = torch.tensor(data["label"].values)

# 2. Fine-tune BERT
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

training_args = TrainingArguments(
    per_device_train_batch_size=8,
    num_train_epochs=3,
    logging_dir="./logs",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    compute_metrics=lambda preds, labels: {
        "accuracy": (preds[1].argmax(-1) == labels).float().mean().item()
    },
)

trainer.train()


# 3. Discriminator Score
def discriminator_score(original, paraphrased):
    inputs = encode_sentences(original, paraphrased)
    with torch.no_grad():
        outputs = model(**inputs)
        # Get the probability of label=1 (human-written)
        probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
        return probs[0][1].item()


# Test
original_sentence = "Your example original sentence here."
paraphrased_sentence = "Your example paraphrased sentence here."

score = discriminator_score(original_sentence, paraphrased_sentence)
print("Discriminator Score:", score)

# BLEU Score


In [6]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.tokenize import word_tokenize


# Tokenization function
def tokenize(sentence):
    return word_tokenize(sentence)


def compute_bleu_score(reference, hypothesis):
    # Tokenize the sentences
    reference_tokens = [tokenize(reference)]
    hypothesis_tokens = tokenize(hypothesis)

    # Use smoothing function
    smoothie = SmoothingFunction().method4

    return sentence_bleu(
        reference_tokens, hypothesis_tokens, smoothing_function=smoothie
    )


# Test
original_sentence = "This is an example sentence."
paraphrased_sentence = "This sentence is an example."

bleu_score = compute_bleu_score(original_sentence, paraphrased_sentence)
print("BLEU Score:", bleu_score)

BLEU Score: 0.27799668259661947


# METEOR Score


In [8]:
import nltk
from nltk.translate.meteor_score import single_meteor_score
from nltk.tokenize import word_tokenize

# Ensure you've downloaded the Punkt tokenizer models
nltk.download("punkt")


def compute_meteor_score(reference, hypothesis):
    # Tokenize the sentences
    reference_tokens = word_tokenize(reference)
    hypothesis_tokens = word_tokenize(hypothesis)

    # The METEOR function in nltk expects the reference as a string and the hypothesis as a list of tokens
    return single_meteor_score(reference_tokens, hypothesis_tokens)


# Test
original_sentence = "This is an example sentence."
paraphrased_sentence = "This sentence is an example."

meteor_score = compute_meteor_score(original_sentence, paraphrased_sentence)
print("METEOR Score:", meteor_score)

METEOR Score: 0.8518518518518519


[nltk_data] Downloading package punkt to /home/vscode/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Toxicity ranking


In [10]:
from detoxify import Detoxify

# each model takes in either a string or a list of strings

results = Detoxify("original").predict("fuck you")

print(results)

{'toxicity': 0.9977558, 'severe_toxicity': 0.45780033, 'obscene': 0.9929531, 'threat': 0.0037068268, 'insult': 0.953166, 'identity_attack': 0.015627943}
